# Import data

In [1]:
import pandas as pd
import numpy as np

ins_df = pd.read_csv('data/instagram_data.csv')
ins_df

,imagename,postid,dataset,dataset.1,postdate,Contents,url,Q1_pertain_to_covid,Q2_cetegory,Q2A_Type of Human,...,Q8_threat_covid,Q9_susceptibility_covid,Q9A_Asian responsible for the covid,Q10_solution_present,Q11_recommended_solution,misinformation,Q12 Presence of conspiracy theory,Q12-Others,Q13. Image of plague doctor costume,Note
0,B-A5FIIIEKJ,5592,middle,2,3/21/20,#covid #covid2020 #covidvirus #virus #coronava...,https://www.instagram.com/p/B-A5FIIIEKJ/,1,1,1,...,1,1,2,1,1,0,2,99,2.0,Stay home
1,B-A74YQn_4a,5593,middle,2,3/21/20,Well this is the final mural of my trip in Aus...,https://www.instagram.com/p/B-A74YQn_4a/,1,1,1,...,1,1,2,2,99,0,2,99,2.0,NaN
2,B-AajnDp6GQ,5575,middle,2,3/21/20,Chegamos !!! Vão seguindo o movimento... Tem m...,https://www.instagram.com/p/B-AajnDp6GQ/,2,99,99,...,99,99,99,99,99,0,99,99,99.0,NaN
3,B-AarR4pUQA,5576,middle,2,3/21/20,😻😻😻😻😻,https://www.instagram.com/p/B-AarR4pUQA/,2,99,99,...,99,99,99,99,99,0,99,99,99.0,NaN
4,B-AE1y3HD-Z,5550,middle,2,3/21/20,EN MI DOMICILIO 🏡\r\r\r\r\n#quedateencasa\r\r\...,https://www.instagram.com/p/B-AE1y3HD-Z/,1,1,1,...,1,1,2,1,1,0,2,99,2.0,mask; Stay home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9643,CEZyheBgtMd,8402,second,3,8/27/20,#water #foryou #followforfollowback #photograp...,https://www.instagram.com/p/CEZyheBgtMd/,2,99,99,...,99,99,99,99,99,0,99,99,99.0,NaN
9644,CEZYoyRluKG,8352,second,3,8/27/20,#like4likes #20likes #tagforlikes #instalikes ...,https://www.instagram.com/p/CEZYoyRluKG/,2,99,99,...,99,99,99,99,99,0,99,99,99.0,NaN
9645,CEZz-solaUF,8403,second,3,8/27/20,🙈🥰😍👉🏽 @love_serie_karma #daancorona #daancoron...,https://www.instagram.com/p/CEZz-solaUF/,2,99,99,...,99,99,99,99,99,0,99,99,99.0,NaN
9646,CEZZ4P4j3rh,8354,second,3,8/27/20,💥🚨 𝗙𝗔𝗟𝗟 𝗜𝗦 𝗖𝗢𝗠𝗜𝗡𝗚 🚨💥⁣\r\r\r\r\r\n⁣\r\r\r\r\r\n...,https://www.instagram.com/p/CEZZ4P4j3rh/,2,99,99,...,99,99,99,99,99,0,99,99,99.0,NaN


In [2]:
from sklearn.utils import shuffle

anger_df = pd.read_csv('data/twitter/anger.tsv', sep='\t').drop(columns=['index', 'intensity'])
fear_df = pd.read_csv('data/twitter/fear.tsv', sep='\t').drop(columns=['index', 'intensity'])
joy_df = pd.read_csv('data/twitter/joy.tsv', sep='\t').drop(columns=['index', 'intensity'])
sadness_df = pd.read_csv('data/twitter/sadness.tsv', sep='\t').drop(columns=['index', 'intensity'])

emotion_df = pd.concat([anger_df, fear_df, joy_df, sadness_df])
emotion_df = shuffle(emotion_df)

emotion_df

,tweet,category
563,"Pops are joyless, soulless toys which look nea...",sadness
28,The moment of the day when you have to start t...,sadness
692,@GucciShade @KellyannePolls @ABC So? Who care...,fear
651,"@eachus At least he's willing to discuss, bett...",joy
268,Optimism leads to success. - Bill Kerr @Coach_...,joy
...,...,...
587,LMAO this person must hate me so bad to talk s...,fear
285,@Sami_3499 Oh noooo! #nomorehammocks #nightma...,fear
496,@HillaryClinton Yeah U gotta make them HEEL. I...,anger
7,#BridgetJonesBaby is the best thing I've seen ...,joy


# Feature Engineering

In [3]:
!pip install transformers datasets

In [18]:
# feature engineering and data augmentation/cleaning. Since we are dealing with tweets (textual data), maybe explore TFIDF

import torch
from transformers import AutoTokenizer
MODEL = "cardiffnlp/twitter-roberta-base-sentiment" # Will likely change
tokenizer = AutoTokenizer.from_pretrained(MODEL)

# Note: How we preprocess may depend on model we use to transfer. 
# This comes from https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Load data into numpy arrays
X = np.array(emotion_df['tweet'])
Y = np.array(emotion_df['category'])

# Preprocess text
for i in range(len(X)):
    X[i] = preprocess(X[i])

# Split into train/val/test sets
TRAIN_PCT, VAL_PCT, TEST_PCT  = 0.6, 0.2, 0.2
train_idx = int(TRAIN_PCT * len(X))
val_idx = train_idx + int(VAL_PCT * len(X))

X_train, Y_train = X[:train_idx], Y[:train_idx]
X_val, Y_val = X[train_idx:val_idx], Y[train_idx:val_idx]
X_test, Y_test = X[val_idx:], Y[val_idx:]

# Tokenize the data
X_train_enc = tokenizer(list(X_train), return_tensors='pt', padding=True, truncation=True)
X_val_enc = tokenizer(list(X_val), return_tensors='pt', padding=True, truncation=True)
X_test_enc = tokenizer(list(X_test), return_tensors='pt', padding=True, truncation=True)

X_train_enc

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': tensor([[    0,   510,  5090,  ...,     1,     1,     1],
        [    0,   133,  1151,  ...,     1,     1,     1],
        [    0,  1039, 12105,  ...,     1,     1,     1],
        ...,
        [    0, 10431, 25441,  ...,     1,     1,     1],
        [    0,  1039, 12105,  ...,     1,     1,     1],
        [    0,  1039, 12105,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

# Model Definition

In [72]:
# TODO: define our machine learning model, from our discussion it we can try deep learning models

# Training

In [73]:
# TODO: train our model using the loaded data

# Performance Evaluation

In [75]:
# TODO: evaluate using accuracy to see how well our model is performing on the test set

# Prediction

In [76]:
# TODO: predict the submission data 